# import Jason file

In [1]:
import json
with open('/Users/rowenahan/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login["api-key"],timeout_s=5.0)
yelp_api

## Search location and term

In [4]:
search_results = yelp_api.search_query(location = "Oahu, Hawaii",
                                      term= "Pizza")
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
search_results["total"]

924

### Import pandas

In [9]:
import pandas as pd


In [10]:
biz=pd.DataFrame(search_results["businesses"])
biz.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,b7beu_ddBb7yiMc0Tgfr0A,old-town-pizzeria-and-tap-house-wahiawa,Old Town Pizzeria & Tap House,https://s3-media3.fl.yelpcdn.com/bphoto/TYq3l6...,False,https://www.yelp.com/biz/old-town-pizzeria-and...,39,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 21.4935665206445, 'longitude': -1...",[],$$,"{'address1': '167 S Kamehameha Hwy', 'address2...",+18082005800,(808) 200-5800,6806.093112
1,ev-HilLF1ru2S76NkAxQIA,uncles-ny-deli-pizza-pasta-and-subs-wahiawa,"Uncle's NY Deli, Pizza, Pasta & Subs",https://s3-media4.fl.yelpcdn.com/bphoto/myqY1l...,False,https://www.yelp.com/biz/uncles-ny-deli-pizza-...,334,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 21.49506, 'longitude': -158.05007}",[pickup],$$,"{'address1': '1718 Wilikina Dr', 'address2': '...",+18085173232,(808) 517-3232,8978.617509
